# Data Prep
In this notebook you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This notebook will have you explore the cardinalities, number of missing values, detect outliers, handle missing values and outliers, and create data quality report for original and cleaned dataset. Additionally, you will be asked to provide documentation for your functions.

## Step 1: Downloading the Data

This notebook will only be using [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in NB1 we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). We will need to read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

You will be utilizing a set of extracted descriptive features much like what you were asked to construct in NB1. However, this dataset contains many more extracted features than you were asked to compute for NB1 (>800), so we need to explore the data to find data quality issues and identify ways to address these issues. Below are links to the full extracted feature dataset for partition 1 and a toy dataset to use for testing you functions.

__Note:__ Since the full dataset, and multiple copies of partially processed intermediary results, tend to take up a bit of space, you can use the toy dataset to implement and test your code. You may need to edit the data to fully test each of the requirements, but that is left as an exercise for the student. The full partition dataset is only included for those who wish to work with it once they have their code implemented. 

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Now that you have the extracted features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.  


In [ ]:
import os
import pandas as pd
from pandas import DataFrame 
import numpy as np

### Read the dataset

In [ ]:
data_dir = 'data/MVTS'
data_file = 'toy_partition1ExtractedFeatures.csv'
full_data_file = 'partition1ExtractedFeatures.csv'

In [ ]:
abt = pd.read_csv(os.path.join(data_dir, data_file), index_col=0)
abt2 = pd.read_csv(os.path.join(data_dir, full_data_file), index_col=0)

### P1: Quality Report

Write a function to extract the various pieces of a data quality report, for a specific attribute, and return a dataframe with this information.

 * 'Feature Name': Contains the time series statistical feature name
 
 * 'Cardinality': Contains the count of unique values for the feature
            
 * 'Non-null Count': Contains the number of non-null entries for the feature
            
 * 'Null Count': Contains the number of null or missing entries for the feature
            
 * 'Min': Contains the minimum value of the feature (Without considering the null or nan value)
 
 * '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
 
 * 'Mean': Contains the mean of the feature values (Without considering the null/nan value)
 
 * '50th': Contains the median of the feature values (Without considering the null/nan value)
            
 * '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
 
 * 'Max': Contains the maximum value of the feature (Without considering the null/nan value),
            
 * 'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
 
In addition to the values above, you should identify the number of upper and lower outliers using the $val < Q1-1.5IQR$ and $val > Q3+1.5IQR$ outlier identification method. These added features should be called `Outlier Count Low` and `Outliers Count High` respectively.


 
 Some useful functions for this can be found at:
 
 * [Numpy.percentile](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)
 
 * [pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html)
 
 * [Numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
 
 * [Numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
 

In [ ]:
def calc_summary_for(feature_name:str, data:DataFrame) -> DataFrame:
    """
    Calculates Summary Features in list of 'summary_feature_names'.

    Parameters
    ----------
    feature_name : str
        Name of feature for summary to be calculated for. required
    data : pandas.DataFrame object
        A DataFrame object containg column named feature_name. required

    Returns : DataFrame object of summary calculated features. 
    """
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    
    # Create DataFrame to return and list at loc to DataFram
    frame2return = pd.DataFrame(columns=summary_feature_names)
    list2add = []
    
    # Find claculated features that have bult in functions
    list2add.append(feature_name)
    list2add.append(data.shape[0])
    list2add.append(data[feature_name].notnull().sum())
    list2add.append(data[feature_name].isnull().sum())
    list2add.append(data[feature_name].min())
    list2add.append(data[feature_name].quantile(q=0.25))
    list2add.append(data[feature_name].mean())

    # Save for Calcuating IQR
    list2add.append(data[feature_name].quantile(q=0.5))
    Q1 = data[feature_name].quantile(q=0.5)
    list2add.append(data[feature_name].quantile(q=0.75))
    Q3 = data[feature_name].quantile(q=0.75)

    list2add.append(data[feature_name].max())
    list2add.append(data[feature_name].std())
    # Find IQR
    IQR = Q3 - Q1

    # Find Range for outliers
    outerBoundLow = Q1 - (1.5 * IQR)
    outerBoundHigh = Q3 + (1.5 * IQR)
    

    # Find calculated features using IQR counting outliers
    countLow = 0
    countHigh = 0
    countInBounds = 0
    for i in data[feature_name]:
        if i < outerBoundLow:
            countLow +=1
        elif i > outerBoundHigh:
            countHigh +=1
        else :
            countInBounds +=1
    
    list2add.append(countLow)
    list2add.append(countHigh)

    # Add list to Dataframe and return it
    frame2return.loc[len(frame2return)] = list2add
    return frame2return

In [ ]:
summary_TOTUSJH_min = calc_summary_for('TOTUSJH_min', abt)
print(summary_TOTUSJH_min) # Used for testing return from calc_summary_for()

### P2: Report Construction
Using what you produced to answere P1, you should now write a function to construct the data quality report for all of the numerical features of our dataset.  You should loop over all of the features in the analytics base table represented by the input feature dataset files from partition 1, with the exception of the first column (this is the index column if you read the file correctly), and the `id`, `lab`, `st`, and `et` columns.  

Your output from this function will be a DataFrame that has 1 row for each feature. 

In [ ]:
def construct_quality_report(data:DataFrame) -> DataFrame:
    """
    Uses calc_summary_for() to constuct full DataFrame of summary features for all features except 'excluded_columns'.

    Parameters
    ----------
    data : pandas.DataFrame object
        A DataFrame object with feature names as columns. required

    Returns : DataFrame object of full quality summary calculated features. 
    """

    excluded_columns = ['id', 'lab', 'st', 'et']
    
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    
    # Create DataFrame to return
    frame2return = pd.DataFrame(columns=summary_feature_names)

    # For each feature calc_summary_feat()
    for col in data.head():
        if col in excluded_columns:
            continue
        df2add = calc_summary_for(col, data)
        frame2return = pd.concat([frame2return,df2add])
    

    return frame2return

In [ ]:
summary_table = construct_quality_report(abt)
print(summary_table.shape)  # checking the dimensionality is often a useful test.

### P3: Drop features with improper cardinality:
Using the quality report summary table that is returned from the function you wrote for P2, we are now going to investigate our data. For this, you should use the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Since we are using real valued features, a majority of them shall have a cardinality close to the sample count. So, for this question, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the feature that have a cardinality less than 10. This feature should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

A useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [ ]:
def drop_low_card_data(summary_table:DataFrame, data:DataFrame) -> None:
    """
    Drops features with Cardinality less than 10.

    Parameters
    ----------
    summary_table : pandas.DataFrame object
        A DataFrame object containing calculated summary features. required
    data : pandas.DataFrame object
        A DataFrame object containing feature names as columns. required

    Returns : Inplace Dataframe Objects with dropped features. 
    """

    # For each Feature check Cardinality 
    for feature in summary_table['Feature Name']:
        if feature in str(summary_table['Feature Name']):      
            tempdf = summary_table[summary_table['Feature Name'] == feature]
            card = tempdf['Cardinality']

            # If Cardinality less than 10 remove from summary 
            if int(card) < 10:
                summary_table.drop(feature, inplace=True)
                data.drop(columns=feature, inplace=True)
        continue

    return summary_table

In [ ]:
drop_low_card_data(summary_table, abt)
print(summary_table.shape)

### P4: Drop features with excessive NaN

Again, using the quality report summary table that is returned from the function you wrote for P2, we are going to continue investigating our data. For this, you should still be using the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Like the features that were dropped for P3, some of the extracted features don't work on all of the variates of the input multi-variate time series samples very well.  So, some of these features return an excessive number of `NaN` values.  These are not verry useful features, so we want to get rid of them before we continue. To do this, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the features that have **more than 1%** of the entries as null/nan values. Again, these features should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

As in P3, a useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [ ]:
def drop_excessive_nan_data(summary_table:DataFrame, data:DataFrame) -> None:
    """
    Drops features with Null Count less than 1% of Cardinality.

    Parameters
    ----------
    summary_table : pandas.DataFrame object
        A DataFrame object containing calculated summary features. required
    data : pandas.DataFrame object
        A DataFrame object containing feature names as columns. required

    Returns : Inplace Dataframe Objects with dropped features. 
    """
    
    # For each Feature check Null Count against Cardinlaity
    for feature in summary_table['Feature Name']:
        if feature in str(summary_table['Feature Name']):      
            tempdf = summary_table[summary_table['Feature Name'] == feature]
            card = int(tempdf['Cardinality'])
            null_count = int(tempdf['Null Count'])

            # If Null Count > 1% Cardinality than  remove from summary 
            if null_count > (.01 * card):
                summary_table.drop(feature, inplace=True)
                data.drop(columns=feature, inplace=True)
        continue

    return summary_table
    

In [ ]:
drop_excessive_nan_data(summary_table, abt)
print(summary_table.shape)

Now that you have the data cleaned up a little, save the results of both your summary table and your analytics base table using the [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method. We will want to use these results for the next notebook.

In [ ]:
out_dir = 'data/MVTS/' # Edited For Unix Machine 
out_summary_table_name = 'data_quality_table.csv'
out_abt_name = 'cleaned_partition1ExtractedFeatures.csv'

# Testing Full Data
summary_table2 = construct_quality_report(abt2)
drop_low_card_data(summary_table2, abt2)
drop_excessive_nan_data(summary_table2, abt2)


In [ ]:
# Output Summary and Cleaned Data CSVs
summary_table.to_csv(out_dir + out_summary_table_name)
abt2.to_csv(out_dir + out_abt_name)